In [1]:
import numpy as np
import tensorflow as tf
import os

In [2]:
import matplotlib.pyplot as plt
import _pickle
import time

In [3]:
# todo
# compare likelihoods
# test with less popular features
# use different priors
# multiclass logistic regression significance tests
# fit mixture of gaussian vs gaussian
# check if weight activity killed
# tansfer learning for new class and robustness

In [4]:
file = open('mean_diff_saliency_mask_vanilla_temp1.pkl', "rb")
mean_diff_saliencies = _pickle.load(file) 
file = open('mean.pkl', "rb")
gaussian_means = _pickle.load(file) 
file = open('covariance.pkl', "rb")
gaussian_covariances = _pickle.load(file)

with open('turker_prob.pkl', "rb") as handle:
    turkers = _pickle.load(handle) 

with open('test_set_17_227.pkl', "rb") as handle:
    test_set = _pickle.load(handle) 
    
with open('turker_count.pkl', "rb") as handle:
    turker_count= _pickle.load(handle) 

In [5]:
male_actors = ['Daniel Radcliffe', 'Gerard Butler', 'Michael Vartan', 'Alec Baldwin', 'Steve Carell',\
                    'Bill Hader', 'Fran Drescher', 'Matt Damon', 'Nicolas Cage']
female_actors = ['Cheryl Hines', 'Selena Gomez', 'Angie Harmon', 'Lorraine Bracco', 'Kristin Chenoweth',\
                      'America Ferrera', 'Anne Hathaway', 'Jennifer Aniston']
gender ={}
for actor in male_actors:
    gender[actor] = 'male'
for actor in female_actors:
    gender[actor] = 'female' 

In [6]:
# creating priors by gender
# features = ['beard','cheek', 'ears', 'eye brows', 'eyes', 'hairline', 'lips', 'moustache', 'nose']
features = ['beard', 'eye brows', 'eyes', 'hairline', 'lips', 'moustache', 'nose']
male_prior = {}
for feature in features:
    male_prior[feature] = 0
for actor in male_actors:
    for stats in turker_count[actor]:
        for feature in features:
            try:
                male_prior[feature] += stats[feature]
            except:
                continue
                

# creating priors by gender
# features = ['beard','cheek', 'ears', 'eye brows', 'eyes', 'hairline', 'lips', 'moustache', 'nose']
female_prior = {}
for feature in features:
    female_prior[feature] = 0
for actor in female_actors:
    for stats in turker_count[actor]:
        for feature in features:
            try:
                female_prior[feature] += stats[feature]
            except:
                continue


prior = {}
for feature in features:
    prior[feature] = male_prior[feature] + female_prior[feature]

total_count = sum(prior.values())
for feature in prior:
    prior[feature] /= total_count
    
total_count = sum(male_prior.values())
for feature in male_prior:
    male_prior[feature] /= total_count
    
total_count = sum(female_prior.values())
for feature in female_prior:
    female_prior[feature] /= total_count 

In [7]:
# monte carlo rectangles generation
def generate_rectangles(means, covariances):
    mc_samples = {}
    for feature in features:
#         print(feature)
        rectangles = []
        while len(rectangles) < 100:
#             print(len(rectangles))
            width, height, y_centre, x_centre = np.random.multivariate_normal(means[feature], \
                                                                              covariances[feature])
            x_0= int(np.round(x_centre - width/2)) 
            x_1= int(np.round(x_centre + width/2)) 
            y_0= int(np.round(226 - y_centre - height/2)) 
            y_1= int(np.round(226 - y_centre + height/2)) 
            
#             x_0= int(np.round(x_centre - height/2)) 
#             x_1= int(np.round(x_centre + height/2)) 
#             y_0= int(np.round(y_centre - width/2)) 
#             y_1= int(np.round(y_centre + width/2)) 
            
#             print(np.array([x_centre, y_centre, height, width]))
#             print(np.array([x_0,x_1,y_0,y_1]))
            if 0<= x_0 < x_1 <=226 and 0<= y_0 < y_1 <=226:
                rectangles.append(np.array([x_0,x_1,y_0,y_1]))
                
        mc_samples[feature] = np.stack(rectangles)
        
    return mc_samples 

mc_rectangle_samples = generate_rectangles(gaussian_means, gaussian_covariances)

In [8]:
def likelihood(rectangles, saliency_map):
    scores = {}
    for feature in features:
        intensity_ratio_sum = 0
        for rec in rectangles[feature]:
            highlight = saliency_map[rec[2]:rec[3], rec[0]:rec[1]]
            intensity_ratio = np.sum(highlight)/np.sum(saliency_map)
            intensity_ratio_sum += intensity_ratio
        scores[feature] = intensity_ratio_sum/len(rectangles[feature])
    
    sum_scores = np.sum(np.array(list(scores.values())))
    probs = {}
    
    for feature in features:
        probs[feature] = scores[feature]/sum_scores
    
    return probs

In [9]:
def posterior(rectangles, saliency_map):
    scores = {}
    for feature in features:
        intensity_ratio_sum = 0
        for rec in rectangles[feature]:
            highlight = saliency_map[rec[2]:rec[3], rec[0]:rec[1]]
            intensity_ratio = np.sum(highlight)/np.sum(saliency_map)
            intensity_ratio_sum += intensity_ratio
        scores[feature] = prior[feature]*intensity_ratio_sum/len(rectangles[feature])
    
    sum_scores = np.sum(np.array(list(scores.values())))
    probs = {}
    
    for feature in sorted(scores.keys()):
        probs[feature] = scores[feature]/sum_scores
    
    return probs

In [10]:
def gender_posterior(rectangles, saliency_map, gender):
    scores = {}
    for feature in features:
        intensity_ratio_sum = 0
        for rec in rectangles[feature]:
            highlight = saliency_map[rec[2]:rec[3], rec[0]:rec[1]]
            intensity_ratio = np.sum(highlight)/np.sum(saliency_map)
            intensity_ratio_sum += intensity_ratio
        if gender == 'male':
            scores[feature] = male_prior[feature]*intensity_ratio_sum/len(rectangles[feature])
        else:
            scores[feature] = female_prior[feature]*intensity_ratio_sum/len(rectangles[feature])
        
    sum_scores = np.sum(np.array(list(scores.values())))
    probs = {}
    
    for feature in sorted(scores.keys()):
        probs[feature] = scores[feature]/sum_scores
    
    return probs

In [11]:
predictions = {}
for actor in sorted(mean_diff_saliencies.keys()):
    predictions[actor]=[]
    for saliency_map in mean_diff_saliencies[actor]:
        probs = likelihood(mc_rectangle_samples, saliency_map)
        predictions[actor].append(probs)

In [12]:
output = open('robust_flat_prior_predictions.pkl', 'wb')
_pickle.dump(predictions, output)

In [13]:
posterior_predictions = {}
for actor in sorted(mean_diff_saliencies.keys()):
    posterior_predictions[actor]=[]
    for saliency_map in mean_diff_saliencies[actor]:
        probs = posterior(mc_rectangle_samples, saliency_map)
        posterior_predictions[actor].append(probs)

In [14]:
gender_posterior_predictions = {}
for actor in sorted(mean_diff_saliencies.keys()):
    gender_posterior_predictions[actor]=[]
    for saliency_map in mean_diff_saliencies[actor]:
        probs = gender_posterior(mc_rectangle_samples, saliency_map, gender[actor])
        gender_posterior_predictions[actor].append(probs)

In [172]:
from collections import OrderedDict
import operator

folder = os.path.join('robust posterior predictions reduced','flat_prior','raw')

count = 0
for actor in sorted(test_set.keys()):
    f, axes = plt.subplots(15, 3, figsize=(45,150))
    for index in range(len(test_set[actor])):
        
        
        ax = axes[index]
        ax[0].axis('off')
#         axes[0].set_size_inches(5, 5)
        #      plt.title("overall actor saliency", fontsize=14)
        ax[0].imshow(test_set[actor][index].astype('uint8'))
        
        ax[1].axis('off')
#         plt.title("overall actor saliency", fontsize=14)
#         axes[1].figure.set_size_inches(10, 10)
        ax[1].imshow(mean_diff_saliencies[actor][index])
        
        prediction = predictions[actor][index]
        turker_data = turkers[actor][index]

        X = np.arange(len(prediction))
        
#         ax = plt.subplot(1,3,3)
        sorted_prediction = sorted(prediction.items(), key=operator.itemgetter(1))
        sorted_prediction.reverse()
        
        ax[2].bar(X, [prediction[key[0]] for key in sorted_prediction],\
               width=0.2, color='b', align='center', label='Prediction')
        ax[2].bar(X-0.2, [turker_data[key[0]] for key in sorted_prediction],\
               width=0.2, color='g', align='center', label='Turkers')
        ax[2].legend()
        ax[2].set_xticks(X)
        ax[2].set_xticklabels([key[0] for key in sorted_prediction])
        ax[2].set_title(actor +"_{}".format(index), fontsize=17)
    print(actor)
    plt.savefig(os.path.join(folder, '{}.png'.format(count)))
    count += 1
    plt.close()

Alec Baldwin
America Ferrera
Angie Harmon
Anne Hathaway
Bill Hader
Cheryl Hines
Daniel Radcliffe
Fran Drescher
Gerard Butler
Jennifer Aniston
Kristin Chenoweth
Lorraine Bracco
Matt Damon
Michael Vartan
Nicolas Cage
Selena Gomez
Steve Carell


In [174]:
folder = os.path.join('robust posterior predictions reduced','prior','raw')

count = 0
for actor in sorted(test_set.keys()):
    f, axes = plt.subplots(15, 3, figsize=(45,150))
    for index in range(len(test_set[actor])):
        
        
        ax = axes[index]
        ax[0].axis('off')
#         axes[0].set_size_inches(5, 5)
        #      plt.title("overall actor saliency", fontsize=14)
        ax[0].imshow(test_set[actor][index].astype('uint8'))
        
        ax[1].axis('off')
#         plt.title("overall actor saliency", fontsize=14)
#         axes[1].figure.set_size_inches(10, 10)
        ax[1].imshow(mean_diff_saliencies[actor][index])
        
        prediction = posterior_predictions[actor][index]
        turker_data = turkers[actor][index]

        X = np.arange(len(prediction))
        
#         ax = plt.subplot(1,3,3)
        sorted_prediction = sorted(prediction.items(), key=operator.itemgetter(1))
        sorted_prediction.reverse()
        
        ax[2].bar(X, [prediction[key[0]] for key in sorted_prediction],\
               width=0.2, color='b', align='center', label='Prediction')
        ax[2].bar(X-0.2, [turker_data[key[0]] for key in sorted_prediction],\
               width=0.2, color='g', align='center', label='Turkers')
        ax[2].legend()
        ax[2].set_xticks(X)
        ax[2].set_xticklabels([key[0] for key in sorted_prediction])
        ax[2].set_title(actor +"_{}".format(index), fontsize=17)
    print(actor)
    plt.savefig(os.path.join(folder, '{}.png'.format(count)))
    count += 1
    plt.close()

Alec Baldwin
America Ferrera
Angie Harmon
Anne Hathaway
Bill Hader
Cheryl Hines
Daniel Radcliffe
Fran Drescher
Gerard Butler
Jennifer Aniston
Kristin Chenoweth
Lorraine Bracco
Matt Damon
Michael Vartan
Nicolas Cage
Selena Gomez
Steve Carell


In [175]:
folder = os.path.join('robust posterior predictions reduced','gender_prior','raw')

count = 0
for actor in sorted(test_set.keys()):
    f, axes = plt.subplots(15, 3, figsize=(45,150))
    for index in range(len(test_set[actor])):
        
        
        ax = axes[index]
        ax[0].axis('off')
#         axes[0].set_size_inches(5, 5)
        #      plt.title("overall actor saliency", fontsize=14)
        ax[0].imshow(test_set[actor][index].astype('uint8'))
        
        ax[1].axis('off')
#         plt.title("overall actor saliency", fontsize=14)
#         axes[1].figure.set_size_inches(10, 10)
        ax[1].imshow(mean_diff_saliencies[actor][index])
        
        prediction = gender_posterior_predictions[actor][index]
        turker_data = turkers[actor][index]

        X = np.arange(len(prediction))
        
#         ax = plt.subplot(1,3,3)
        sorted_prediction = sorted(prediction.items(), key=operator.itemgetter(1))
        sorted_prediction.reverse()
        
        ax[2].bar(X, [prediction[key[0]] for key in sorted_prediction],\
               width=0.2, color='b', align='center', label='Prediction')
        ax[2].bar(X-0.2, [turker_data[key[0]] for key in sorted_prediction],\
               width=0.2, color='g', align='center', label='Turkers')
        ax[2].legend()
        ax[2].set_xticks(X)
        ax[2].set_xticklabels([key[0] for key in sorted_prediction])
        ax[2].set_title(actor +"_{}".format(index), fontsize=17)
    print(actor)
    plt.savefig(os.path.join(folder, '{}.png'.format(count)))
    count += 1
    plt.close()

Alec Baldwin
America Ferrera
Angie Harmon
Anne Hathaway
Bill Hader
Cheryl Hines
Daniel Radcliffe
Fran Drescher
Gerard Butler
Jennifer Aniston
Kristin Chenoweth
Lorraine Bracco
Matt Damon
Michael Vartan
Nicolas Cage
Selena Gomez
Steve Carell


In [176]:
folder = os.path.join('robust prior predictions reduced','gender_prior','raw')

count = 0
for actor in sorted(test_set.keys()):
    f, axes = plt.subplots(15, 3, figsize=(45,150))
    for index in range(len(test_set[actor])):
        
        
        ax = axes[index]
        ax[0].axis('off')
#         axes[0].set_size_inches(5, 5)
        #      plt.title("overall actor saliency", fontsize=14)
        ax[0].imshow(test_set[actor][index].astype('uint8'))
        
        ax[1].axis('off')
#         plt.title("overall actor saliency", fontsize=14)
#         axes[1].figure.set_size_inches(10, 10)
        ax[1].imshow(mean_diff_saliencies[actor][index])
        
#         prediction = gender_posterior_predictions[actor][index]
        turker_data = turkers[actor][index]

       
        
#         ax = plt.subplot(1,3,3)
        
        if gender[actor] == 'male':
            prediction = male_prior
            sorted_prediction = sorted(male_prior.items(), key=operator.itemgetter(1))
            
        else:
            prediction = female_prior
            sorted_prediction = sorted(female_prior.items(), key=operator.itemgetter(1))
            
        sorted_prediction.reverse()
        X = np.arange(len(prediction))
        ax[2].bar(X, [prediction[key[0]] for key in sorted_prediction],\
               width=0.2, color='b', align='center', label='Prediction')
        ax[2].bar(X-0.2, [turker_data[key[0]] for key in sorted_prediction],\
               width=0.2, color='g', align='center', label='Turkers')
        ax[2].legend()
        ax[2].set_xticks(X)
        ax[2].set_xticklabels([key[0] for key in sorted_prediction])
        ax[2].set_title(actor +"_{}".format(index), fontsize=17)
    print(actor)
    plt.savefig(os.path.join(folder, '{}.png'.format(count)))
    count += 1
    plt.close()

Alec Baldwin
America Ferrera
Angie Harmon
Anne Hathaway
Bill Hader
Cheryl Hines
Daniel Radcliffe
Fran Drescher
Gerard Butler
Jennifer Aniston
Kristin Chenoweth
Lorraine Bracco
Matt Damon
Michael Vartan
Nicolas Cage
Selena Gomez
Steve Carell


In [11]:
# creating priors by gender
# features = ['beard','cheek', 'ears', 'eye brows', 'eyes', 'hairline', 'lips', 'moustache', 'nose']
male_prior = {}
for feature in features:
    male_prior[feature] = 0
for actor in male_actors:
    for stats in turker_count[actor][:10]:
        for feature in features:
            try:
                male_prior[feature] += stats[feature]
            except:
                continue
                

# creating priors by gender
# features = ['beard','cheek', 'ears', 'eye brows', 'eyes', 'hairline', 'lips', 'moustache', 'nose']
female_prior = {}
for feature in features:
    female_prior[feature] = 0
for actor in female_actors:
    for stats in turker_count[actor][:10]:
        for feature in features:
            try:
                female_prior[feature] += stats[feature]
            except:
                continue


prior = {}
for feature in features:
    prior[feature] = male_prior[feature] + female_prior[feature]

total_count = sum(prior.values())
for feature in prior:
    prior[feature] /= total_count
    
total_count = sum(male_prior.values())
for feature in male_prior:
    male_prior[feature] /= total_count
    
total_count = sum(female_prior.values())
for feature in female_prior:
    female_prior[feature] /= total_count 

In [12]:
predictions = {}
for actor in sorted(mean_diff_saliencies.keys()):
    predictions[actor]=[]
    for saliency_map in mean_diff_saliencies[actor][10:]:
        probs = likelihood(mc_rectangle_samples, saliency_map)
        predictions[actor].append(probs)

In [13]:
import operator
# check likelihood and accuracy
loglik = 0
accurate_count = 0
count = 0
const  = 1E-20
for actor in predictions:
#     loglik = 0
#     accurate_count = 0
#     count = 0
#     const  = 1E-20
    for i in range(5):
        prediction = predictions[actor][i]
        order = sorted(prediction.items(), key=operator.itemgetter(1))
        data = turker_count[actor][10+i]
        for feature in prediction:
            loglik += data[feature]*np.log(prediction[feature]+const)
        
        accurate_count +=  data[order[-1][0]]
        count += sum(data.values())
# print(actor)
print('log-likelihood {}'.format(loglik))
print('accuracy {}'.format(accurate_count/count))

log-likelihood -3869.3607018597254
accuracy 0.29442738043001315


In [14]:
gender_posterior_predictions = {}
for actor in sorted(mean_diff_saliencies.keys()):
    gender_posterior_predictions[actor]=[]
    for saliency_map in mean_diff_saliencies[actor][10:]:
        probs = gender_posterior(mc_rectangle_samples, saliency_map, gender[actor])
        gender_posterior_predictions[actor].append(probs)

In [15]:
# check likelihood and accuracy
loglik = 0
accurate_count = 0
count = 0
const  = 1E-20
for actor in gender_posterior_predictions:
#     loglik = 0
#     accurate_count = 0
#     count = 0
#     const  = 1E-20
    for i in range(5):
        prediction = gender_posterior_predictions[actor][i]
        order = sorted(prediction.items(), key=operator.itemgetter(1))
        data = turker_count[actor][10+i]
        for feature in prediction:
            loglik += data[feature]*np.log(prediction[feature]+const)
        
        accurate_count +=  data[order[-1][0]]
        count += sum(data.values())
# print(actor)
print('log-likelihood {}'.format(loglik))
print('accuracy {}'.format(accurate_count/count))

log-likelihood -3458.1388878869734
accuracy 0.3189995612110575


In [20]:
# check likelihood and accuracy
loglik = 0
accurate_count = 0
count = 0
const  = 1E-20
for actor in gender_posterior_predictions:
#     loglik = 0
#     accurate_count = 0
#     count = 0
#     const  = 1E-20
    for i in range(5):
        if gender[actor] == 'male':
            prediction = male_prior
        else:
            prediction = female_prior
        order = sorted(prediction.items(), key=operator.itemgetter(1))
        data = turker_count[actor][10+i]
        for feature in prediction:
            loglik += data[feature]*np.log(prediction[feature]+const)
        
        accurate_count +=  data[order[-1][0]]
        count += sum(data.values())

#     print(actor)
print('log-likelihood {}'.format(loglik))
print('accuracy {}'.format(accurate_count/count))

log-likelihood -3198.5590185138285
accuracy 0.3181219833260202


In [16]:
# main problems are with turker data
# maybe imbalanced, also tend to label similar feature
# maybe should try new set of actors who have very distinct features to do anchoring
# people choocse what seems to get highlighted in that image, but not what is salient about the person
# probaly better to frame questionas as what is salient about this individual

In [17]:
# creating priors by gender
features = ['beard','cheek', 'ears', 'eye brows', 'eyes', 'hairline', 'lips', 'moustache', 'nose']
male_prior = {}
for feature in features:
    male_prior[feature] = 0
for actor in male_actors:
    for stats in turker_count[actor]:
        for feature in features:
            try:
                male_prior[feature] += stats[feature]
            except:
                continue
                

# creating priors by gender
features = ['beard','cheek', 'ears', 'eye brows', 'eyes', 'hairline', 'lips', 'moustache', 'nose']
female_prior = {}
for feature in features:
    female_prior[feature] = 0
for actor in female_actors:
    for stats in turker_count[actor]:
        for feature in features:
            try:
                female_prior[feature] += stats[feature]
            except:
                continue


prior = {}
for feature in features:
    prior[feature] = male_prior[feature] + female_prior[feature]

total_count = sum(prior.values())
for feature in prior:
    prior[feature] /= total_count
    
total_count = sum(male_prior.values())
for feature in male_prior:
    male_prior[feature] /= total_count
    
total_count = sum(female_prior.values())
for feature in female_prior:
    female_prior[feature] /= total_count 

In [39]:
actor_diff_saliencies = {}

for actor in mean_diff_saliencies:
    mean_diff = np.sum(mean_diff_saliencies[actor], axis=0)
    mean_diff = mean_diff/np.max(mean_diff)
    actor_diff_saliencies[actor] = mean_diff

In [40]:
actor_posterior_predictions = {}
for actor in sorted(actor_diff_saliencies.keys()):
    probs = gender_posterior(mc_rectangle_samples, actor_diff_saliencies[actor], gender[actor])
    actor_posterior_predictions[actor]=probs

In [41]:
# creating turker coiunt per actor
actor_stats = {}
features = ['beard','cheek', 'ears', 'eye brows', 'eyes', 'hairline', 'lips', 'moustache', 'nose']
for actor in actor_diff_saliencies:
    actor_stats[actor] = {}
    for feature in features:
        actor_stats[actor][feature] = 0  
    for stats in turker_count[actor]:
        for feature in features:
            try:
                 actor_stats[actor][feature] += stats[feature]
            except:
                continue

for actor in actor_stats:
    total_count = sum(actor_stats[actor].values())
    for feature in features:
        actor_stats[actor][feature] /= total_count

In [44]:
actors = sorted(test_set.keys())

f, axes = plt.subplots(17, 3, figsize=(45,170))

for i in range(len(actors)):
    actor = actors[i]
    ax = axes[i]

    ax[0].set_title(actor)
    ax[0].axis('off')
    ax[0].imshow(actor_diff_saliencies[actor])
        
      
    turker_data = actor_stats[actor]
    sorted_turker_data = sorted(turker_data.items(), key=operator.itemgetter(1))
    sorted_turker_data.reverse()
    
    X = np.arange(len(turker_data))
    ax[1].bar(X, [actor_posterior_predictions[actor][key[0]] for key in sorted_turker_data],\
           width=0.2, color='b', align='center', label='Posterior Prediction')
    ax[1].bar(X-0.2, [turker_data[key[0]] for key in sorted_turker_data],\
           width=0.2, color='g', align='center', label='Turkers')
    ax[1].legend()
    ax[1].set_xticks(X)
    ax[1].set_xticklabels([key[0] for key in sorted_turker_data])
    ax[1].set_title('posterior vs turkers', fontsize=17)
    
    
    if gender[actor] == 'male':
        prediction = male_prior

    else:
        prediction = female_prior

    X = np.arange(len(turker_data))
    ax[2].bar(X, [prediction[key[0]] for key in sorted_turker_data],\
           width=0.2, color='b', align='center', label='Prior Prediction')
    ax[2].bar(X-0.2, [turker_data[key[0]] for key in sorted_turker_data],\
           width=0.2, color='g', align='center', label='Turkers')
    ax[2].legend()
    ax[2].set_xticks(X)
    ax[2].set_xticklabels([key[0] for key in sorted_turker_data])
    ax[2].set_title('prior vs turkers', fontsize=17)
    print(actor)
plt.savefig('actor_predictions.png')
plt.close()

Alec Baldwin
America Ferrera
Angie Harmon
Anne Hathaway
Bill Hader
Cheryl Hines
Daniel Radcliffe
Fran Drescher
Gerard Butler
Jennifer Aniston
Kristin Chenoweth
Lorraine Bracco
Matt Damon
Michael Vartan
Nicolas Cage
Selena Gomez
Steve Carell


In [45]:
actor_bp_predictions = {}
for actor in sorted(actor_diff_saliencies.keys()):
    probs = likelihood(mc_rectangle_samples, actor_diff_saliencies[actor])
    actor_bp_predictions[actor]=probs

In [47]:
actors = sorted(test_set.keys())

f, axes = plt.subplots(17, 3, figsize=(45,170))

for i in range(len(actors)):
    actor = actors[i]
    ax = axes[i]

    ax[0].set_title(actor)
    ax[0].axis('off')
    ax[0].imshow(actor_diff_saliencies[actor])
        
      
    turker_data = actor_stats[actor]
    sorted_turker_data = sorted(turker_data.items(), key=operator.itemgetter(1))
    sorted_turker_data.reverse()
    
    X = np.arange(len(turker_data))
    ax[1].bar(X, [actor_bp_predictions[actor][key[0]] for key in sorted_turker_data],\
           width=0.2, color='b', align='center', label='Posterior Prediction')
    ax[1].bar(X-0.2, [turker_data[key[0]] for key in sorted_turker_data],\
           width=0.2, color='g', align='center', label='Turkers')
    ax[1].legend()
    ax[1].set_xticks(X)
    ax[1].set_xticklabels([key[0] for key in sorted_turker_data])
    ax[1].set_title('posterior vs turkers', fontsize=17)
    
#     if gender[actor] == 'male':
#         prediction = male_prior

#     else:
#         prediction = female_prior

    X = np.arange(len(turker_data))
    ax[2].bar(X, [float(1/9) for key in sorted_turker_data],\
           width=0.2, color='b', align='center', label='Prior Prediction')
    ax[2].bar(X-0.2, [turker_data[key[0]] for key in sorted_turker_data],\
           width=0.2, color='g', align='center', label='Turkers')
    ax[2].legend()
    ax[2].set_xticks(X)
    ax[2].set_xticklabels([key[0] for key in sorted_turker_data])
    ax[2].set_title('prior vs turkers', fontsize=17)
    print(actor)
plt.savefig('actor_bp_predictions.png')
plt.close()

Alec Baldwin
America Ferrera
Angie Harmon
Anne Hathaway
Bill Hader
Cheryl Hines
Daniel Radcliffe
Fran Drescher
Gerard Butler
Jennifer Aniston
Kristin Chenoweth
Lorraine Bracco
Matt Damon
Michael Vartan
Nicolas Cage
Selena Gomez
Steve Carell


In [63]:
output = open('actor_bp_predictions.pkl', 'wb')
_pickle.dump(actor_bp_predictions, output)

In [84]:
from scipy.stats import spearmanr
cors = []
pvals = []
for actor in sorted(test_set.keys()):
    print(actor)
    turker_data = actor_stats[actor]
    sorted_turker_data = sorted(turker_data.items(), key=operator.itemgetter(1))
    predicted = np.array([actor_bp_predictions[actor][key[0]] for key in sorted_turker_data])
    turker = np.array([turker_data[key[0]] for key in sorted_turker_data])
    cor, pval = spearmanr(predicted, turker) 
    cors.append(cor)
    pvals.append(pval)
    print(spearmanr(predicted, turker))
print('correlation')
print(np.mean(np.array(cors)))    
print('pvalue')
print(np.mean(np.array(pvals)))

Alec Baldwin
SpearmanrResult(correlation=0.88148255470705361, pvalue=0.0016774219366218158)
America Ferrera
SpearmanrResult(correlation=0.84757937952601303, pvalue=0.0039085480832251867)
Angie Harmon
SpearmanrResult(correlation=0.77824949014282241, pvalue=0.013505972810006874)
Anne Hathaway
SpearmanrResult(correlation=0.6134670389756931, pvalue=0.078915226437158675)
Bill Hader
SpearmanrResult(correlation=0.94928890506913455, pvalue=9.2000228099956688e-05)
Cheryl Hines
SpearmanrResult(correlation=0.48535989707831934, pvalue=0.18535381786420974)
Daniel Radcliffe
SpearmanrResult(correlation=0.76666666666666672, pvalue=0.015944016578974009)
Fran Drescher
SpearmanrResult(correlation=0.72242227221614608, pvalue=0.027924260269707922)
Gerard Butler
SpearmanrResult(correlation=0.59999999999999998, pvalue=0.087622829041402395)
Jennifer Aniston
SpearmanrResult(correlation=0.71196667880185094, pvalue=0.031425255860766076)
Kristin Chenoweth
SpearmanrResult(correlation=0.55940239048716867, pvalue=0.

In [68]:
with open('saliency_data17/true_class_saliency_vanilla_temp1.pkl', "rb") as handle:
    true_saliencies = _pickle.load(handle)

In [71]:
actor_saliencies = {}
for actor in true_saliencies:
    mean_true = np.sum(true_saliencies[actor], axis=0)
    mean_true = mean_true/np.max(mean_true)
    actor_saliencies[actor] = mean_true

In [75]:
actor_true_bp_predictions = {}
for actor in sorted(actor_saliencies.keys()):
    probs = likelihood(mc_rectangle_samples, actor_saliencies[actor])
    actor_true_bp_predictions[actor]=probs

In [76]:
actors = sorted(test_set.keys())

f, axes = plt.subplots(17, 3, figsize=(45,170))

for i in range(len(actors)):
    actor = actors[i]
    ax = axes[i]

    ax[0].set_title(actor)
    ax[0].axis('off')
    ax[0].imshow(actor_saliencies[actor])
        
      
    turker_data = actor_stats[actor]
    sorted_turker_data = sorted(turker_data.items(), key=operator.itemgetter(1))
    sorted_turker_data.reverse()
    
    X = np.arange(len(turker_data))
    ax[1].bar(X, [actor_true_bp_predictions[actor][key[0]] for key in sorted_turker_data],\
           width=0.2, color='b', align='center', label='Posterior Prediction')
    ax[1].bar(X-0.2, [turker_data[key[0]] for key in sorted_turker_data],\
           width=0.2, color='g', align='center', label='Turkers')
    ax[1].legend()
    ax[1].set_xticks(X)
    ax[1].set_xticklabels([key[0] for key in sorted_turker_data])
    ax[1].set_title('posterior vs turkers', fontsize=17)
    
#     if gender[actor] == 'male':
#         prediction = male_prior

#     else:
#         prediction = female_prior

    X = np.arange(len(turker_data))
    ax[2].bar(X, [float(1/9) for key in sorted_turker_data],\
           width=0.2, color='b', align='center', label='Prior Prediction')
    ax[2].bar(X-0.2, [turker_data[key[0]] for key in sorted_turker_data],\
           width=0.2, color='g', align='center', label='Turkers')
    ax[2].legend()
    ax[2].set_xticks(X)
    ax[2].set_xticklabels([key[0] for key in sorted_turker_data])
    ax[2].set_title('prior vs turkers', fontsize=17)
    print(actor)
plt.savefig('actor_true_bp_predictions.png')
plt.close()

Alec Baldwin
America Ferrera
Angie Harmon
Anne Hathaway
Bill Hader
Cheryl Hines
Daniel Radcliffe
Fran Drescher
Gerard Butler
Jennifer Aniston
Kristin Chenoweth
Lorraine Bracco
Matt Damon
Michael Vartan
Nicolas Cage
Selena Gomez
Steve Carell


In [82]:
from scipy.stats import spearmanr
cors = []
pvals = []
for actor in sorted(test_set.keys()):
    print(actor)
    turker_data = actor_stats[actor]
    sorted_turker_data = sorted(turker_data.items(), key=operator.itemgetter(1))
    predicted = np.array([actor_true_bp_predictions[actor][key[0]] for key in sorted_turker_data])
    turker = np.array([turker_data[key[0]] for key in sorted_turker_data])
    cor, pval = spearmanr(predicted, turker) 
    cors.append(cor)
    pvals.append(pval)
    print(spearmanr(predicted, turker))

print('correlation')
print(np.mean(np.array(cors)))    
print('pvalue')
print(np.mean(np.array(pvals)))

Alec Baldwin
SpearmanrResult(correlation=0.93233731747861426, pvalue=0.00024822089814914041)
America Ferrera
SpearmanrResult(correlation=0.77977302916393199, pvalue=0.013205186732060017)
Angie Harmon
SpearmanrResult(correlation=0.77824949014282241, pvalue=0.013505972810006874)
Anne Hathaway
SpearmanrResult(correlation=0.6975036196572949, pvalue=0.036722324744514964)
Bill Hader
SpearmanrResult(correlation=0.93233731747861426, pvalue=0.00024822089814914041)
Cheryl Hines
SpearmanrResult(correlation=0.64435710474190677, pvalue=0.061032957699099791)
Daniel Radcliffe
SpearmanrResult(correlation=0.84999999999999987, pvalue=0.0037047773275858133)
Fran Drescher
SpearmanrResult(correlation=0.8616843969807042, pvalue=0.0028228814548817384)
Gerard Butler
SpearmanrResult(correlation=0.59999999999999998, pvalue=0.087622829041402395)
Jennifer Aniston
SpearmanrResult(correlation=0.72891826639237112, pvalue=0.025883597169560207)
Kristin Chenoweth
SpearmanrResult(correlation=0.72891826639237112, pvalue=

In [130]:
with open('true_class_saliency_mask_vanilla_temp1.pkl', "rb") as handle:
    true_saliencies = _pickle.load(handle)

In [131]:
actor_saliencies = {}
for actor in true_saliencies:
    mean_true = np.sum(true_saliencies[actor], axis=0)
    mean_true = mean_true/np.max(mean_true)
    actor_saliencies[actor] = mean_true

In [132]:
def likelihood(rectangles, saliency_map):
    scores = {}
    for feature in sorted(rectangles.keys()):
        intensity_ratio_sum = 0
        for rec in rectangles[feature]:
            highlight = saliency_map[rec[2]:rec[3], rec[0]:rec[1]]
            intensity_ratio = np.sum(highlight)/np.sum(saliency_map)
            intensity_ratio_sum += intensity_ratio
        scores[feature] = intensity_ratio_sum/len(rectangles[feature])
    
    sum_scores = np.sum(np.array(list(scores.values())))
    probs = {}
    
    for feature in sorted(scores.keys()):
        probs[feature] = scores[feature]/sum_scores
    
    return probs

In [133]:
actor_true_bp_predictions = {}
for actor in sorted(actor_saliencies.keys()):
    probs = likelihood(mc_rectangle_samples, actor_saliencies[actor])
    actor_true_bp_predictions[actor]=probs

In [134]:
actors = sorted(test_set.keys())

f, axes = plt.subplots(17, 3, figsize=(45,170))

for i in range(len(actors)):
    actor = actors[i]
    ax = axes[i]

    ax[0].set_title(actor)
    ax[0].axis('off')
    ax[0].imshow(actor_saliencies[actor])
        
      
    turker_data = actor_stats[actor]
    sorted_turker_data = sorted(turker_data.items(), key=operator.itemgetter(1))
    sorted_turker_data.reverse()
    
    X = np.arange(len(turker_data))
    ax[1].bar(X, [actor_true_bp_predictions[actor][key[0]] for key in sorted_turker_data],\
           width=0.2, color='b', align='center', label='Posterior Prediction')
    ax[1].bar(X-0.2, [turker_data[key[0]] for key in sorted_turker_data],\
           width=0.2, color='g', align='center', label='Turkers')
    ax[1].legend()
    ax[1].set_xticks(X)
    ax[1].set_xticklabels([key[0] for key in sorted_turker_data])
    ax[1].set_title('posterior vs turkers', fontsize=17)
    
#     if gender[actor] == 'male':
#         prediction = male_prior

#     else:
#         prediction = female_prior

    X = np.arange(len(turker_data))
    ax[2].bar(X, [float(1/9) for key in sorted_turker_data],\
           width=0.2, color='b', align='center', label='Prior Prediction')
    ax[2].bar(X-0.2, [turker_data[key[0]] for key in sorted_turker_data],\
           width=0.2, color='g', align='center', label='Turkers')
    ax[2].legend()
    ax[2].set_xticks(X)
    ax[2].set_xticklabels([key[0] for key in sorted_turker_data])
    ax[2].set_title('prior vs turkers', fontsize=17)
    print(actor)
plt.savefig('actor_true_bp_predictions.png')
plt.close()

Alec Baldwin
America Ferrera
Angie Harmon
Anne Hathaway
Bill Hader
Cheryl Hines
Daniel Radcliffe
Fran Drescher
Gerard Butler
Jennifer Aniston
Kristin Chenoweth
Lorraine Bracco
Matt Damon
Michael Vartan
Nicolas Cage
Selena Gomez
Steve Carell


In [135]:
from scipy.stats import spearmanr
cors = []
pvals = []
for actor in sorted(test_set.keys()):
    print(actor)
    turker_data = actor_stats[actor]
    sorted_turker_data = sorted(turker_data.items(), key=operator.itemgetter(1))
    predicted = np.array([actor_true_bp_predictions[actor][key[0]] for key in sorted_turker_data])
    turker = np.array([turker_data[key[0]] for key in sorted_turker_data])
    cor, pval = spearmanr(predicted, turker) 
    cors.append(cor)
    pvals.append(pval)
    print(spearmanr(predicted, turker))

print('correlation')
print(np.mean(np.array(cors)))    
print('pvalue')
print(np.mean(np.array(pvals)))

Alec Baldwin
SpearmanrResult(correlation=0.93233731747861426, pvalue=0.00024822089814914041)
America Ferrera
SpearmanrResult(correlation=0.84757937952601303, pvalue=0.0039085480832251867)
Angie Harmon
SpearmanrResult(correlation=0.76151294196770791, pvalue=0.017117125525800674)
Anne Hathaway
SpearmanrResult(correlation=0.6975036196572949, pvalue=0.036722324744514964)
Bill Hader
SpearmanrResult(correlation=0.93233731747861426, pvalue=0.00024822089814914041)
Cheryl Hines
SpearmanrResult(correlation=0.74477639379259353, pvalue=0.021319728123320163)
Daniel Radcliffe
SpearmanrResult(correlation=0.90000000000000002, pvalue=0.00094306232234032928)
Fran Drescher
SpearmanrResult(correlation=0.8616843969807042, pvalue=0.0028228814548817384)
Gerard Butler
SpearmanrResult(correlation=0.66666666666666674, pvalue=0.049867230568885111)
Jennifer Aniston
SpearmanrResult(correlation=0.72891826639237112, pvalue=0.025883597169560207)
Kristin Chenoweth
SpearmanrResult(correlation=0.7628214415734117, pvalue

In [142]:
with open('mean_diff_saliency_mask_vanilla_temp1_20.pkl', "rb") as handle:
    true_saliencies = _pickle.load(handle)

In [143]:
actor_saliencies = {}
for actor in true_saliencies:
    mean_true = np.sum(true_saliencies[actor], axis=0)
    mean_true = mean_true/np.max(mean_true)
    actor_saliencies[actor] = mean_true

In [144]:
def likelihood(rectangles, saliency_map):
    scores = {}
    for feature in sorted(rectangles.keys()):
        intensity_ratio_sum = 0
        for rec in rectangles[feature]:
            highlight = saliency_map[rec[2]:rec[3], rec[0]:rec[1]]
            intensity_ratio = np.sum(highlight)/np.sum(saliency_map)
            intensity_ratio_sum += intensity_ratio
        scores[feature] = intensity_ratio_sum/len(rectangles[feature])
    
    sum_scores = np.sum(np.array(list(scores.values())))
    probs = {}
    
    for feature in sorted(scores.keys()):
        probs[feature] = scores[feature]/sum_scores
    
    return probs

In [145]:
actor_true_bp_predictions = {}
for actor in sorted(actor_saliencies.keys()):
    probs = likelihood(mc_rectangle_samples, actor_saliencies[actor])
    actor_true_bp_predictions[actor]=probs

In [146]:
actors = sorted(test_set.keys())

f, axes = plt.subplots(17, 3, figsize=(45,170))

for i in range(len(actors)):
    actor = actors[i]
    ax = axes[i]

    ax[0].set_title(actor)
    ax[0].axis('off')
    ax[0].imshow(actor_saliencies[actor])
        
      
    turker_data = actor_stats[actor]
    sorted_turker_data = sorted(turker_data.items(), key=operator.itemgetter(1))
    sorted_turker_data.reverse()
    
    X = np.arange(len(turker_data))
    ax[1].bar(X, [actor_true_bp_predictions[actor][key[0]] for key in sorted_turker_data],\
           width=0.2, color='b', align='center', label='Posterior Prediction')
    ax[1].bar(X-0.2, [turker_data[key[0]] for key in sorted_turker_data],\
           width=0.2, color='g', align='center', label='Turkers')
    ax[1].legend()
    ax[1].set_xticks(X)
    ax[1].set_xticklabels([key[0] for key in sorted_turker_data])
    ax[1].set_title('posterior vs turkers', fontsize=17)
    
#     if gender[actor] == 'male':
#         prediction = male_prior

#     else:
#         prediction = female_prior

    X = np.arange(len(turker_data))
    ax[2].bar(X, [float(1/9) for key in sorted_turker_data],\
           width=0.2, color='b', align='center', label='Prior Prediction')
    ax[2].bar(X-0.2, [turker_data[key[0]] for key in sorted_turker_data],\
           width=0.2, color='g', align='center', label='Turkers')
    ax[2].legend()
    ax[2].set_xticks(X)
    ax[2].set_xticklabels([key[0] for key in sorted_turker_data])
    ax[2].set_title('prior vs turkers', fontsize=17)
    print(actor)
plt.savefig('mean_diff_robust_actor_bp_predictions.png')
plt.close()

Alec Baldwin
America Ferrera
Angie Harmon
Anne Hathaway
Bill Hader
Cheryl Hines
Daniel Radcliffe
Fran Drescher
Gerard Butler
Jennifer Aniston
Kristin Chenoweth
Lorraine Bracco
Matt Damon
Michael Vartan
Nicolas Cage
Selena Gomez
Steve Carell


In [147]:
from scipy.stats import spearmanr
cors = []
pvals = []
for actor in sorted(test_set.keys()):
    print(actor)
    turker_data = actor_stats[actor]
    sorted_turker_data = sorted(turker_data.items(), key=operator.itemgetter(1))
    predicted = np.array([actor_true_bp_predictions[actor][key[0]] for key in sorted_turker_data])
    turker = np.array([turker_data[key[0]] for key in sorted_turker_data])
    cor, pval = spearmanr(predicted, turker) 
    cors.append(cor)
    pvals.append(pval)
    print(spearmanr(predicted, turker))

print('correlation')
print(np.mean(np.array(cors)))    
print('pvalue')
print(np.mean(np.array(pvals)))

Alec Baldwin
SpearmanrResult(correlation=0.84757937952601303, pvalue=0.0039085480832251867)
America Ferrera
SpearmanrResult(correlation=0.7628214415734117, pvalue=0.016814014277908546)
Angie Harmon
SpearmanrResult(correlation=0.66109365291702116, pvalue=0.05252286046708899)
Anne Hathaway
SpearmanrResult(correlation=0.70590727772545503, pvalue=0.03357930729576155)
Bill Hader
SpearmanrResult(correlation=0.93233731747861426, pvalue=0.00024822089814914041)
Cheryl Hines
SpearmanrResult(correlation=0.44351852664053321, pvalue=0.2317840652993742)
Daniel Radcliffe
SpearmanrResult(correlation=0.84999999999999987, pvalue=0.0037047773275858133)
Fran Drescher
SpearmanrResult(correlation=0.8616843969807042, pvalue=0.0028228814548817384)
Gerard Butler
SpearmanrResult(correlation=0.45000000000000001, pvalue=0.22421610749233659)
Jennifer Aniston
SpearmanrResult(correlation=0.52549921530612809, pvalue=0.14624025831231621)
Kristin Chenoweth
SpearmanrResult(correlation=0.5085476277156078, pvalue=0.162108